https://colab.research.google.com/github/fastai/fastbook/blob/master/02_production.ipynb#scrollTo=cNzVuQIQUsQX

In [3]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [4]:
#hide
from fastbook import *
from fastai.vision.widgets import *



In [5]:
!pip install -Uqq duckduckgo_search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 18.4 MB/s eta 0:00:00


In [ ]:
from itertools import islice
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    # DDGS().images return an interator containing all images found
    # we use islice to limit the number of results returned
    return L(islice(DDGS().images(term), max_images)).itemgot('image')

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('grizzly', max_images=1)
urls[0]


In [ ]:
from fastdownload import download_url
dest = 'grizzly.jpg'
download_url(urls[0], dest, show_progress=True)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')
for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    results=search_images(f'{o} bear')
    download_images(dest, urls=results)
    #resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
fns = get_image_files(path)
len(fns)


Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    #item_tfms=[Resize(192, method='squish')]
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
).dataloaders(path, bs=32)

bears.show_batch(max_n=10)

In [ ]:
learn = vision_learner(bears, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner

doc(ImageClassifierCleaner)

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()


In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn = vision_learner(bears, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
learn.export()

In [ ]:
path=Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf=load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('grizzly.jpg')


In [ ]:
learn_inf.dls.vocab

In [ ]:
#hide_output
btn_upload=widgets.FileUpload()
btn_upload

In [ ]:
#hide
btn_upload=SimpleNamespace(data=['grizzly.jpg'])

In [ ]:
img=PILImage.create(btn_upload.data[-1])

In [ ]:
# Display the image using matplotlib
plt.imshow(img)
plt.axis('off')  # Hide axes for cleaner display
plt.show()


In [ ]:
#hide_output
out_pl=widgets.Output()
out_pl.clear_output()
with out_pl:display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs=learn_inf.predict(img)

In [ ]:
#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
#hide_output
btn_run = widgets.Button(description='Classify')
btn_run


In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)


In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()


In [1]:
#hide_output
VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])


NameError: name 'VBox' is not defined

In [ ]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila
